In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(1)

TIME_STEP=10
INPUT_SIZE=1
LR=0.02
DOWNLOAD_MNIST=True

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn=nn.RNN(
            input_size=1,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Linear(32, 1)
        
    def forward(self, x, h_state):
        # x: (batch, time_step, input_size)
        # h_state: (n_layers, batch, hidden_size)
        # r_out: (batch, time_step, hidden_size)
        
#         r_out, h_state=self.rnn(x, h_state)
        
#         print(x.size(), h_state.size(), r_out.size())
        
#         outs=[]
        
#         for time_step in range(r_out.size(1)):
#             outs.append(self.out(r_out[:, time_step, :]))
#         return torch.stack(outs, dim=1), h_state

        r_out, h_state=self.rnn(x, h_state)
        print("x: ", x.size(), "r_out: ", r_out.size(), "h_state: ", h_state.size())
        r_out=r_out.view(-1, 32)
        outs=self.out(r_out)
        # out: (time_step, batch)
        print("r_out: ", r_out.size(), "outs: ", outs.size())
        return outs.view(-1, TIME_STEP, 1), h_state
    
rnn=RNN()
print(rnn)

In [ ]:
h_state=None

start, end=1*np.pi, (1+1)*np.pi

steps=np.linspace(start, end, TIME_STEP, dtype=np.float32, endpoint=False)
x_np=np.sin(steps)
y_np=np.cos(steps)

x=torch.from_numpy(x_np[np.newaxis, :, np.newaxis])
y=torch.from_numpy(y_np[np.newaxis, :, np.newaxis])

prediction, h_state=rnn(x, h_state)
# h_state=h_state.data
loss=loss_func(prediction, y)

optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
optimizer=torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func=nn.MSELoss()

h_state=None

plt.figure(1, figsize=(12, 5))
plt.ion()

for step in range(100):
    start, end=step*np.pi, (step+1)*np.pi
    
    steps=np.linspace(start, end, TIME_STEP, dtype=np.float32, endpoint=False)
    x_np=np.sin(steps)
    y_np=np.cos(steps)
    
    x=torch.from_numpy(x_np[np.newaxis, :, np.newaxis])
    y=torch.from_numpy(y_np[np.newaxis, :, np.newaxis])
    
    prediction, h_state=rnn(x, h_state)
    h_state=h_state.data
    
    loss=loss_func(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    plt.plot(steps, y_np, 'r-')
    plt.plot(steps, prediction.data.numpy().flatten(), 'b-')
    plt.draw()
    # plt.pause(0.05)
    
plt.ioff()
plt.show()
    